### 신경망의 톱니바퀴: 텐서 연산
- 심층 신경망이 학습한 모든 변환을 수치 데이터 텐서에 적용하는 몇 종류의 텐서 연산(tensor operation)으로 나타낼 수 있다.
    - 예) 텐서 덧셈, 텐서 곱셈

---
첫 번째 예제에서는 Dense 층을 쌓아서 신경망을 만들었다 케라스의 층은 다음과 같이 생성한다
-  keras.layer.Dense(512, activation='relu')

- 이 층은 2D 텐서를 입력받고 입력 텐서의 새로운 표현인 또 다른 2D텐서를 반환하는 함수처럼 해석할 수 있다
- 구체적으로 보면 이 함수는 다음과 같다

output = relu(dot(W, input) + b) (W는 2D텐서, b는 벡터, 둘은 모두 층의 속성)
- 여기에는 3개의 텐서 연산이 있다
    1. 입력 텐서와 텐서 W 사이의 점곱(dot)
    2. 점곱의 결과인 2D 텐서와 벡터 b 사이의 덧셈(+)
    3. relu(렐루) 연산 relu는 max(x, 0)
         - 렐루 함수는 입력이 0보다 크면 입력을 그대로 반환하고 0보다 작으면 0을 반환한다

---
### 원소별 연산
- relu 함수와 덧셈은 원소별 연산(element-wise operation)입니다
- 이 연산은 텐서에 있는 각 원소에 독립적으로 적용된다

 파이썬으로 단순한 원소별 연산을 구현한다면 다음 relu 연산 구현처럼 for 반복문을 사용할 것이다

In [29]:
def naive_relu(x):
    assert len(x.shape) == 2 # x는 2D 넘파이 배열
    
    x = x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] = max (x[i,j],0)
    return x
    

In [30]:
def naive_add(x, y):
    assert len(x.shape)
    assert x.shape == y.shape # x,y는 2D 넘파이 배열
    
    x = x.copy() # 입력 텐서 자체를 바꾸지 않도록 복사
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] += y[i,j]
    return x

같은 원리로 원소별 곱셈,뺄셈 등을 할 수 있다

넘파이 배열을 다룰 때는 최적화된 넘파이 내장 함수로 이런 연산들을 처리할 수 있다  
넘파이는 시스템에 설치된 BLAS 구현에 복잡한 일들을 위임한다  
BLAS는 고도의 병렬화되고 효율적인 저수준의 텐서 조작 루틴이며, 전형적으로 포트란이나 C언어로 구현되어 있다  
넘파이는 다음과 같은 원소별 연산을 빠르게 처리한다  

---
z = x + y == 원소별 덧셈  
z = np.maximum(z,0.) == 원소별 렐루 함수

---
### 브로드캐스팅
- 브로드캐스팅은 두단계로 이루어진다
    1. 큰 텐서의 ndim에 맞도록 작은 텐서에(브로드캐스팅 축이라고 부르는)축이 추가 된다.
    2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복된다

X의 크기 (32,10) y의 크기 (10,)바로 가정한다면  
먼저 첫 번째 축을 추가하여 크기를 (1, 10)으로 만든다  
그런 다음 y를 이 축에 32번 반복하면 텐서 Y의 크기는 (32,10) 가 된다  
여기에서 Y[i,:] == y for i in range(0,32)  
이제 X 와 크기가 같으므로 더할 수 있다

---
파이썬으로 단순하게 구성한 예  
  
def naive_add_matrix_and_vector(x, y)  
    assert len(x.shape) == 2 #x는 2D 넘파이 배열  
    assert len(y.shpae) == 1 #y는 넘파이 벡터  
    assert x.shape[1] == y.shape[0]  
    
    x = x.copy() #입력 자체를 바꾸지 않도록 복사  
    for i in range(x.shape[0]):  
        for j in range(x.shape[1]):  
            x[i,j] +=y[j]
    return x

---
크기가 다른 두 텐서에 브로드캐스팅으로 원소별 maximum 연산을 적용하는 예시

In [33]:
import numpy as np

In [36]:
x = np.random.random((64,3,32,10)) # x는 (64,3,32,10)크기의 랜덤 텐서
y = np.random.random((32,10))# y는 (32,10)크기의 랜덤 텐서

z = np.maximum(x,y) # 출력 z의 크기는 x와 동일하게 (64,3,32,10)
z

array([[[[0.85259451, 0.99697912, 0.60131279, ..., 0.51161119,
          0.943053  , 0.97568665],
         [0.56975305, 0.57734184, 0.96954128, ..., 0.56442234,
          0.66253694, 0.47254832],
         [0.87050705, 0.7955805 , 0.26833411, ..., 0.60239919,
          0.59464257, 0.90892488],
         ...,
         [0.60518043, 0.88482414, 0.50977092, ..., 0.86218176,
          0.93506137, 0.6748732 ],
         [0.7014869 , 0.91482837, 0.76939475, ..., 0.48215763,
          0.86617328, 0.20632441],
         [0.71243375, 0.976558  , 0.99407578, ..., 0.86594916,
          0.69580852, 0.66418129]],

        [[0.85259451, 0.99697912, 0.78120555, ..., 0.51161119,
          0.943053  , 0.97568665],
         [0.84120191, 0.57734184, 0.92994085, ..., 0.36220298,
          0.66253694, 0.47254832],
         [0.87050705, 0.34165587, 0.94424727, ..., 0.47291337,
          0.59464257, 0.90892488],
         ...,
         [0.81872148, 0.88482414, 0.51772172, ..., 0.49382487,
          0.49722802, 0.3

---
### 텐서 점곱
- 텐서 곱셈이라고도 부르는(원소별 곱셈과 혼동하지 말자) 점곱 연산 (dot operation)은 가장 널리 사용되고 유용한 텐서 연산이다.
- 원소별 연산과 반대로 입력 텐서의 원소들을 결합 시킨다
- 텐서플로에서는 dot 연산자가 다르지만 넘파이와 케라시는 점곱 연산에 보편적인 dot연산자를 사용한다
- 넘파이,케라스,씨아노,텐서플로에서 원소별 곱셈은 * 연산자를 사용한다

z = np.dot(x,y)  
z = x·y

In [1]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1  # x와 y는 넘파이 벡터입니다.
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]

    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

- 두 벡터의 점곱은 스칼라가 되므로 원소 개수가 같은 벡터끼리 점곱이 가능하다  
- 행렬 x와 벡터 y 사이에서도 점곱이 가능하다
---
y와 x의 행 사이에서 점곱이 일어나므로 벡터가 반환되는 예제

In [2]:
import numpy as np
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2   # x는 넘파이 행렬입니다.
    assert len(y.shape) == 1   # y는 넘파이 벡터입니다.
    assert x.shape[1] == y.shape[0]  # x의 두 번째 차원이 y의 첫 번째 차원과 같아야 합니다!

    z = np.zeros(x.shape[0])   # 이 연산은 x의 행과 같은 크기의 0이 채워진 벡터를 만듭니다.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

행렬 벡터 점곱과 벡터-벡터 점곱 사이의 관계를 부각하기 위해 앞에서 만든 함수를 재사용

In [3]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

두 텐서 중 하나라도 ndim이 1보다 크면 dot 연산에 교환 법칙이 성립되지 않는다  
다시 말하면 dot(x,y)와 dot(y,x)가 같지 않다

---
단순 구현의 예

In [4]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2   # x와 y는 넘파이 행렬입니다.
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]  # x의 두 번째 차원이 y의 첫 번째 차원과 같아야 합니다!

    z = np.zeros((x.shape[0], y.shape[1]))  # 이 연산은 0이 채워진 특정 크기의 벡터를 만듭니다.
    for i in range(x.shape[0]):     # x의 행을 반복합니다.
        for j in range(y.shape[1]): # y의 열을 반복합니다.
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

---
크기를 맞추는 동일한 규칙을 따르면 다음과 같이 고차원 텐서간의 점곱을 할 수 있다.

(a, b, c, d) . (d,) -> (a, b, c) 

(a, b, c, d) . (d, e) -> (a, b, c, e)
 

---
### 2.3.4 텐서 크기 변환
- 첫 번쨰 Dense 층에는 사용되지 않았지만 신경망에 주입할 숫자 데이터를 전처리할 때 사용한다

In [17]:
from keras.datasets import mnist
(train_images, train_labels), (test_images,test_labels) = mnist.load_data()

In [18]:
train_images = train_images.reshape((60000, 28 * 28))


- 텐서의 크기를 변환한다는 것은 특정 크기에 맞게 열과 행을 재배열한다는 뜻
- 크기가 반환된 텐서는 원래 텐서와 원소 개수가 동일하다

In [19]:
x = np.array([[0,1],
              [2,3],
              [4,5]])
x.shape

(3, 2)

In [20]:
x = x.reshape((6, 1))
x

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5]])

In [21]:
x = x.reshape((2, 3))
x

array([[0, 1, 2],
       [3, 4, 5]])

---
자주 사용하는 특별한 크기 변환은 전치(transposition)  
행렬의 전치는 행과 열을 바꾸는 것을 의미한다 즉 x[i,:]이 x[:,i]가 된다 

In [25]:
x = np.zeros((300, 20))  # 모두 0으로 채워진 (300, 20) 크기의 행렬을 만듭니다.
x = np.transpose(x)
x.shape

(20, 300)

---
### 2.3.5 텐서 연산의 기하학적 해석
- 텐서 연산이 조작하는 텐서의 내용은 어떤 기하학적 공간에 있는 좌표 포인트로 해석될 수 있기 때문에 모든 텐서 연산은 기하학적 해석이 가능하다.
- 일반적으로 아핀 변환affine transformation 23, 회전, 스케일링scaling 등처럼 기본적인 기하학적 연산은 텐서 연산으로 표현될 수 있습니다. 예를 들어 theta 각도로 2D 벡터를 회전하는 것은 2×2 행렬 R = [u, v]를 점곱하여 구현할 수 있습니다. 여기에서 u, v는 동일 평면상의 벡터이며, u = [cos(theta), sin(theta)]고 v = [-sin(theta), cos(theta)]입니다.

---
### 2.3.6 딥러닝의 기하학적 해석
- 신경망은 전체적으로 텐서 연산의 연결로 구성된 것이고, 모든 텐서 연산은 입력 데이터의 기하학적 변환임을 배웠다.
    - 단순한 단계들이 길게 이어져 구현된 신경망을 고차원 공간에 매우 복잡하게 기하학적 변환을 하는 것으로 해석할 수 있다.